# DUE DILLIGENCE

 ## load env

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# Load API keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# Optional: LangSmith settings (for LangChain traces)
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGSMITH_TRACING", "false")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT", "")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGSMITH_PROJECT", "")

print(openai_api_key[:10])



sk-proj-Xn


# load and extract text from pdf

## extract datat from JSW energy

In [13]:
import json
import warnings
import camelot
import fitz  # PyMuPDF
import re

warnings.filterwarnings("ignore", category=UserWarning)

file_path = "/Users/macbook/Documents/Scraping/notebook-due-diligence/knowledge/JSW Energy - Annual Reports for Indian Subsidiaries.pdf"

def clean_text(text):
    """
    Replace multiple whitespace/newlines with single space and strip edges.
    """
    return re.sub(r'\s+', ' ', text).strip()

def clean_and_structure_table(raw_json_str):
    """
    Converts a JSON string of table rows with numeric keys into a JSON string with proper headers.
    Cleans cell values by removing newlines and trimming spaces.

    Args:
        raw_json_str (str): JSON string representing list of row dicts with keys as string indices ("0", "1", ...).

    Returns:
        str: Pretty-printed JSON string with header-based keys and cleaned cell values.
    """
    rows = json.loads(raw_json_str)
    if not rows:
        return "[]"

    # Extract header row and clean headers
    header_row = rows[0]
    header = []
    for i in range(len(header_row)):
        key = str(i)
        cell = header_row.get(key, "").strip()
        if not cell:
            cell = f"Column_{i}"
        header.append(cell)

    # Process remaining rows, clean values
    structured_rows = []
    for row in rows[1:]:
        row_dict = {}
        for i in range(len(header)):
            key = str(i)
            cell_value = row.get(key, "").replace('\n', ' ').strip()
            if cell_value:
                row_dict[header[i]] = cell_value
        if row_dict:
            structured_rows.append(row_dict)

    return json.dumps(structured_rows, indent=2, ensure_ascii=False)

def chunk_table_json(df, max_rows=20):
    """
    Split dataframe into chunks of max_rows rows and convert each chunk to JSON string.

    Args:
        df (pd.DataFrame): Dataframe to chunk.
        max_rows (int): Maximum number of rows per chunk.

    Returns:
        list[str]: List of JSON strings for each chunk.
    """
    chunks = []
    columns = list(df.columns)

    for start_row in range(0, len(df), max_rows):
        chunk_df = df.iloc[start_row:start_row + max_rows]
        rows_as_dicts = []
        for _, row in chunk_df.iterrows():
            row_dict = {col: clean_text(str(row[col])) for col in columns}
            rows_as_dicts.append(row_dict)
        chunk_json_str = json.dumps(rows_as_dicts, ensure_ascii=False)
        chunks.append(chunk_json_str)

    return chunks

def extract_table_chunks(file_path, max_rows=20):
    """
    Extract tables from PDF file, chunk them, clean and structure each chunk.

    Args:
        file_path (str): Path to PDF file.
        max_rows (int): Max rows per chunk.

    Returns:
        list[dict]: List of dicts representing cleaned table chunks.
    """
    tables = camelot.read_pdf(file_path, pages='all')
    print(f"📄 Total tables extracted: {len(tables)}")
    all_chunks = []

    for idx, table in enumerate(tables):
        df = table.df
        chunks = chunk_table_json(df, max_rows)
        for chunk_idx, chunk_json in enumerate(chunks):
            cleaned_chunk_json = clean_and_structure_table(chunk_json)
            all_chunks.append({
                "type": "table",
                "table_id": idx + 1,
                "chunk_id": chunk_idx + 1,
                "text": cleaned_chunk_json
            })
    return all_chunks

def extract_text_chunks(file_path, max_chars=1000):
    """
    Extract text from PDF pages and chunk by max_chars length.

    Args:
        file_path (str): Path to PDF file.
        max_chars (int): Max characters per text chunk.

    Returns:
        list[dict]: List of dicts representing text chunks.
    """
    doc = fitz.open(file_path)
    text_chunks = []

    for page_num, page in enumerate(doc, start=1):
        text = page.get_text()
        cleaned_text = clean_text(text)
        chunks = [cleaned_text[i:i + max_chars] for i in range(0, len(cleaned_text), max_chars)]
        for chunk in chunks:
            text_chunks.append({
                "type": "text",
                "page": page_num,
                "chunk_id": len(text_chunks) + 1,
                "text": chunk
            })

    return text_chunks

# ========== Main execution ==========

text_chunks = extract_text_chunks(file_path)
table_chunks = extract_table_chunks(file_path)

all_chunks = text_chunks + table_chunks

print(f"\n✅ Total text chunks: {len(text_chunks)}")
print(f"✅ Total table chunks: {len(table_chunks)}")
print(f"✅ Total combined chunks: {len(all_chunks)}")

# Preview first chunk
print("\n🔹 First chunk preview:")
print(json.dumps(all_chunks, indent=2, ensure_ascii=False))


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

📄 Total tables extracted: 45

✅ Total text chunks: 52
✅ Total table chunks: 47
✅ Total combined chunks: 99

🔹 First chunk preview:
[
  {
    "type": "text",
    "page": 1,
    "chunk_id": 1,
    "text": "31.03.2016 31.12.2015 31.03.2015 31.03.2016 31.03.2015 Audited Unaudited Audited Audited Audited 1 Income from operation a) Net Sales / Income from Operations 2128,74,998 2128,75,001 2510,58,906 11165,00,000 10191,00,000 b) Other Operating Income - - - - - Total Income from operations (a+b) (net) 2128,74,998 2128,75,001 2510,58,906 11165,00,000 10191,00,000 2 Expenditure : a) Employee Benefits Expenses 22,09,728 22,86,679 17,29,380 90,61,907 80,47,497 b) Depreciation and Amortisation Expenses 729,71,152 737,70,868 723,53,559 2934,73,606 2934,32,832 c) Other Expenses 136,84,102 50,12,841 127,57,876 293,60,735 246,21,360 Total 888,64,982 810,70,388 868,40,816 3318,96,248 3261,01,689 3 Profit / (Loss) from Operations before Other income, Finance costs, and Exceptional items (1 - 2) 1240,1

In [25]:
import json

clean_texts_for_embedding = []

for chunk in all_chunks:
    if chunk.get("type") == "table":
        try:
            rows = json.loads(chunk["text"])
        except json.JSONDecodeError:
            print(f"Warning: Skipping invalid JSON in table chunk_id={chunk.get('chunk_id')}")
            continue

        lines = []

        for row in rows:
            # Skip rows with only one key (likely a section heading)
            if len(row) == 1:
                continue

            parts = [f"{key}: {value}" for key, value in row.items()]
            line = "; ".join(parts)
            lines.append(line)

        cleaned_chunk_text = "\n".join(lines)

        clean_texts_for_embedding.append({
            "text": cleaned_chunk_text,
            "metadata": {
                "chunk_id": chunk.get("chunk_id"),
                "table_id": chunk.get("table_id"),
                "type": "table"
            }
        })

    elif chunk.get("type") == "text":
        clean_texts_for_embedding.append({
            "text": chunk.get("text", ""),
            "metadata": {
                "chunk_id": chunk.get("chunk_id"),
                "type": "text"
            }
        })

# Print all cleaned chunks
if clean_texts_for_embedding:
    print(json.dumps(clean_texts_for_embedding, indent=2))
else:
    print("No chunks processed.")


[
  {
    "text": "31.03.2016 31.12.2015 31.03.2015 31.03.2016 31.03.2015 Audited Unaudited Audited Audited Audited 1 Income from operation a) Net Sales / Income from Operations 2128,74,998 2128,75,001 2510,58,906 11165,00,000 10191,00,000 b) Other Operating Income - - - - - Total Income from operations (a+b) (net) 2128,74,998 2128,75,001 2510,58,906 11165,00,000 10191,00,000 2 Expenditure : a) Employee Benefits Expenses 22,09,728 22,86,679 17,29,380 90,61,907 80,47,497 b) Depreciation and Amortisation Expenses 729,71,152 737,70,868 723,53,559 2934,73,606 2934,32,832 c) Other Expenses 136,84,102 50,12,841 127,57,876 293,60,735 246,21,360 Total 888,64,982 810,70,388 868,40,816 3318,96,248 3261,01,689 3 Profit / (Loss) from Operations before Other income, Finance costs, and Exceptional items (1 - 2) 1240,10,016 1318,04,613 1642,18,090 7846,03,752 6929,98,311 4 Other Income 143,52,380 87,19,931 31,40,919 1971,31,237 65,90,303 5 Profit / (Loss) before Finance costs and Exceptional items (3

## embeddings store in vector db

In [28]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Convert chunks to Document objects (with metadata if available)
documents = [
    Document(page_content=chunk["text"], metadata=chunk.get("metadata", {}))
    for chunk in clean_texts_for_embedding
]

# Initialize the OpenAI embeddings model
embeddings_model = OpenAIEmbeddings()

# Create FAISS vector store directly from documents and embeddings
vectorstore = FAISS.from_documents(documents, embeddings_model)

print(f"Vectorstore contains {vectorstore.index.ntotal} vectors")


Vectorstore contains 99 vectors


## Retrieval QA

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# Assuming vectorstore is already initialized and loaded somewhere
# from langchain.vectorstores import FAISS
# vectorstore = FAISS.load_local(...)

# Load your knowledge files fully as strings
with open("knowledge/creditworthiness_rules.txt", "r") as f:
    creditworthiness_rules = f.read()

with open("knowledge/financial_analysis_guide.txt", "r") as f:
    financial_analysis_guide = f.read()

with open("knowledge/financial_flags.txt", "r") as f:
    financial_flags = f.read()

# query = "What is the current service cost and opening balance, also tell any red or green flags if exist?"
# query = "hat are the key factors affecting the company’s creditworthiness according to the latest report?"
# query = "Explain the financial impact of the current service cost and opening balance on the company’s liabilities."
# query = "Are there any green or red flags in the financial statements for the current fiscal year?"
# query = "Summarize the financial analysis guide points relevant to investment risk assessment."
query = "What do the financial flags indicate about the company's liquidity and solvency?"

# Retrieve relevant docs with scores
# Note: similarity_search returns docs, but does not return scores by default.
# Use similarity_search_with_score() if your vectorstore supports it (e.g., FAISS)

results_with_scores = vectorstore.similarity_search_with_score(query, k=3)

print(f"Top {len(results_with_scores)} documents retrieved for query: '{query}'\n")
context_texts = []
for i, (doc, score) in enumerate(results_with_scores, 1):
    print(f"Document {i} (score: {score:.4f}):")
    print(doc.page_content[:500], "...")  # Print first 500 chars for brevity
    print("Metadata:", doc.metadata)
    print("-" * 60)
    context_texts.append(doc.page_content)

# Prepare system prompt combining your static files
system_prompt = f"""
You are a helpful financial analyst assistant. Based on the provided documents,
answer the question in a clear, concise, and narrative style. Use complete sentences,
and explain financial terms where necessary. Use the following knowledge to answer user queries accurately.

Creditworthiness Rules:
{creditworthiness_rules}

Financial Analysis Guide:
{financial_analysis_guide}

Financial Flags:
{financial_flags}

Answer in a clear, narrative way.
"""

# Combine retrieved docs content for context
combined_context = "\n\n".join(context_texts)

# Compose chat messages with system prompt and user question plus context
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=f"Context:\n{combined_context}\n\nQuestion: {query}")
]

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Generate response
response = llm(messages)

print("\n\nAnswer:\n", response.content)


Top 3 documents retrieved for query: 'What do the financial flags indicate about the company's liquidity and solvency?'

Document 1 (score: 0.3942):
Standard 3 on “Cash Flow Statements” and presents the cash flows by operating, investing and financing activities of the Company, Cash and Cash equivalents presented in the Cash Flow Statement consist of cash on hand and demand deposits with banks. ...
Metadata: {'chunk_id': 30, 'type': 'text'}
------------------------------------------------------------
Document 2 (score: 0.4459):
rtain as the case may be to be realized. Tax credit is recognized in respect of Minimum Alternate Tax (MAT) paid in terms of Section 115JAA of the Income Tax Act, 1961 based on convincing evidence that the Company will pay normal income tax within the statutory time frame and the same is reviewed at each balance sheet date. (xiii) EARNINGS PER SHARE: The Company reports basic and diluted Earnings Per Share (EPS) in accordance with Accounting Standard 20 on “Earn